# Data Import from NCBI GenBank


In [41]:
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "jhwnkim@mit.edu"  # Always tell NCBI who you are

# Read Covid19 reference sequence
ref = SeqIO.read("./data/ref_sequence.gb", "genbank")
print('Reference Covid sequence')
print(ref.id)
print(repr(ref.seq))
print(len(ref.seq))


# Read downloaded sequence file from NCBI GenBank Virus site
records = list( SeqIO.parse("./data/MA-sequences.fasta", "fasta") )

def get_meta(record_id='NC_045512.2'): # default is the reference
    handle = Entrez.efetch(db="nuccore", id=record_id, rettype="gb", retmode="xml")
    fetch_record = Entrez.read(handle)    
    handle.close()
    
    metadata = {
        'id': record_id,
        'date': fetch_record[0]['GBSeq_create-date'],
    }
    
    for qualifier in fetch_record[0]['GBSeq_feature-table'][0]['GBFeature_quals']:
        if qualifier['GBQualifier_name'] == 'country':
            metadata['country'] = qualifier['GBQualifier_value']

    print(metadata)
    return metadata
    
for record in records:
    get_meta(record.id)


Reference Covid sequence
NC_045512.2
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')
29903
{'id': 'MZ144330.1', 'date': '07-MAY-2021', 'country': 'USA: Massachusetts'}
{'id': 'MZ144344.1', 'date': '07-MAY-2021', 'country': 'USA: Massachusetts'}
{'id': 'MZ144393.1', 'date': '07-MAY-2021', 'country': 'USA: Massachusetts'}


In [35]:
# examine record

import json

# fetch_record[0].keys()
print(fetch_record[0]['GBSeq_create-date'])
# print(json.dumps(fetch_record[0]['GBSeq_feature-table'][0]['GBFeature_quals'], indent=1))
# print(json.dumps(fetch_record[0]['GBSeq_feature-table'][0], indent=1))

print()


for qualifier in fetch_record[0]['GBSeq_feature-table'][0]['GBFeature_quals']:
    if qualifier['GBQualifier_name'] == 'country':
        print(qualifier['GBQualifier_value'] )
# print(json.dumps(fetch_record[0], indent=1))

13-JAN-2020

China
